In [20]:
spark.sql(
    """
    Drop table if exists PL7G1_GOLD.Doencas_Air_Quality
    """
)

DataFrame[]

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [22]:
air_quality = spark.table("PL7G1_GOLD.Air_Quality_Gases")
hospital = spark.table("PL7G1_GOLD.doencas")

In [23]:
hospital_Ar = hospital.join(air_quality, (hospital.year == air_quality.Year) & (hospital.borough == air_quality.Borough),"inner").drop(hospital.borough)

In [24]:
hospital_Ar =hospital_Ar.drop(air_quality.Year)

In [25]:
hospital_Ar.printSchema()

root
 |-- year: integer (nullable = true)
 |-- asthma: long (nullable = true)
 |-- bronchiolitis: long (nullable = true)
 |-- chronic_pulmonary_disease: long (nullable = true)
 |-- cystic_fibrosis: long (nullable = true)
 |-- ecmo: long (nullable = true)
 |-- alveolar_lung_diseases: long (nullable = true)
 |-- pneumonia: long (nullable = true)
 |-- pulmonary_embolism: long (nullable = true)
 |-- respiratory_failure: long (nullable = true)
 |-- respiratory_malignancy: long (nullable = true)
 |-- bronchitis: long (nullable = true)
 |-- respiratory_system_diagnosis: long (nullable = true)
 |-- tracheostomy: long (nullable = true)
 |-- respiratory_infection: long (nullable = true)
 |-- total: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- PM2_5_SUM: double (nullable = true)
 |-- PM2_5_AVG: double (nullable = true)
 |-- PM2_5_MIN: double (nullable = true)
 |-- PM2_5_MAX: double (nullable = true)
 |-- NO2_SUM: double (nullable = true)
 |-- NO2_AVG: double (nullable = true

In [26]:
spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Doencas_Air_Quality (
        year int,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        asthma long,
        bronchiolitis long,
        chronic_pulmonary_disease long,
        cystic_fibrosis long,
        ecmo long,
        alveolar_lung_diseases long,
        pneumonia long,
        pulmonary_embolism long,
        respiratory_failure long,
        respiratory_malignancy long,
        bronchitis long,
        respiratory_system_diagnosis long,
        tracheostomy long,
        respiratory_infection long,
        total long
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Doencas_Air_Quality'
    """
)

DataFrame[]

In [27]:
hospital_Ar \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true")\
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Doencas_Air_Quality")

In [28]:
spark.table("PL7G1_GOLD.Doencas_Air_Quality").show()

+----+-------------+------------------+------------------+---------+---------+------------------+------------------+-------+-------+------------------+------------------+------+------+------------------+-------------------+-------+-------+------+-------------+-------------------------+---------------+----+----------------------+---------+------------------+-------------------+----------------------+----------+----------------------------+------------+---------------------+-----+
|year|      Borough|         PM2_5_SUM|         PM2_5_AVG|PM2_5_MIN|PM2_5_MAX|           NO2_SUM|           NO2_AVG|NO2_MIN|NO2_MAX|            O3_SUM|            O3_AVG|O3_MIN|O3_MAX|           SO2_SUM|            SO2_AVG|SO2_MIN|SO2_MAX|asthma|bronchiolitis|chronic_pulmonary_disease|cystic_fibrosis|ecmo|alveolar_lung_diseases|pneumonia|pulmonary_embolism|respiratory_failure|respiratory_malignancy|bronchitis|respiratory_system_diagnosis|tracheostomy|respiratory_infection|total|
+----+-------------+------------

In [29]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Doencas_Air_Quality/`
""").show()

++
||
++
++



In [30]:
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Doencas_Air_Quality_presto (
        year int,
        Borough VARCHAR(50),
        PM2_5_SUM Double,
        PM2_5_AVG Double,
        PM2_5_MIN Double,
        PM2_5_MAX Double,
        NO2_SUM Double,
        NO2_AVG Double,
        NO2_MIN Double,
        NO2_MAX Double,
        O3_SUM Double,
        O3_AVG Double,
        O3_MIN Double,
        O3_MAX Double,
        SO2_SUM Double,
        SO2_AVG Double,
        SO2_MIN Double,
        SO2_MAX Double,
        asthma long,
        bronchiolitis long,
        chronic_pulmonary_disease long,
        cystic_fibrosis long,
        ecmo long,
        alveolar_lung_diseases long,
        pneumonia long,
        pulmonary_embolism long,
        respiratory_failure long,
        respiratory_malignancy long,
        bronchitis long,
        respiratory_system_diagnosis long,
        tracheostomy long,
        respiratory_infection long,
        total long
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
    STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
    OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Doencas_Air_Quality/_symlink_format_manifest/'
    """).show()

++
||
++
++



DataFrame[]